# Using the vtkInterface Package

This notebook is a work in progress to demo how PVGeo can be used with `vtki` for creating integrated visualization directly in a Python environment.

In [1]:
import vtki
import PVGeo
from PVGeo.grids import ExtractTopography
from PVGeo.ubc import TopoReader
from PVGeo.filters import PercentThreshold, ManySlicesAlongAxis
import numpy as np
import discretize

## Load the Data

In [2]:
# Load the TensorMesh and some already processed model data 
mesh = discretize.TensorMesh.readUBC('craig_chile.msh', directory='data/Craig-Chile')
models = {'lpout': mesh.readModelUBC(fileName='Lpout.mod', directory='data/Craig-Chile')}

In [3]:
# Load topography data using PVGeo
topo = TopoReader().Apply('data/Craig-Chile/LdM_topo.topo')
topo

(vtkCommonDataModelPython.vtkPolyData)0x12569cfa8

## Build a Pipeline 

Here we build up a pipeline that will transform and integrate the data

In [4]:
extractor = ExtractTopography(offset=-150, tolerance=10, op='underneath')
extracted = extractor.Apply(mesh.toVTK(models=models), topo)

In [5]:
# threshold out the topography
subsurface = PercentThreshold().Apply(extracted, 'Extracted')
# threshold out the high values
low = PercentThreshold(invert=True, percent=35).Apply(subsurface, 'lpout')

In [6]:
# Slice the subsurface model
sliced = ManySlicesAlongAxis(numSlices=5, axis=1).Apply(subsurface)
sliced

(vtkCommonDataModelPython.vtkMultiBlockDataSet)0x12570f108

In [7]:
# Get the scalar range to plot everything the same way
arr = PVGeo.getArray(subsurface, 'lpout')
rng = np.nanmin(arr), np.nanmax(arr)

## Plot in an Interactive 3D Window

In [8]:
p = vtki.Plotter()
# p.add_mesh(extracted, syle='wireframe', opacity=0.25, scalars='lpout', rng=rng)
p.add_bounds_axes(extracted, color='k', fontsize=30)
p.add_mesh(low, scalars='lpout', stitle='lpout', rng=rng)
p.add_mesh(sliced, scalars='lpout', rng=rng)
p.add_mesh(topo, color='white', opacity=0.75)

p.add_axes()
p.plot()

[(313882.32057365216, 6005680.853308033, 765.5176418489975),
 (363625.0, 6007625.0, -1073.0),
 (0.04167821732553165, -0.12949650140842953, 0.990703579444196)]